# AB Testing Course - PA James

## 0.1 Imposts

In [1]:
import pandas as pd
import numpy as np
import math
from statsmodels.stats import api as sms
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_1samp,shapiro,ttest_ind,mannwhitneyu,f_oneway,chi2_contingency
import seaborn as sns

# 1.0 Load Data

In [2]:
df_raw = pd.read_csv('../data/ab_data.csv')

In [3]:
df_raw.shape

(294478, 5)

In [4]:
df_raw.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


# 2.0 Design de Experimentos

## 2.1 Formação das Hipóteses

H0: A conversão da nova página é de 13% <br>
H1: A conversão da nova página é diferente de 13%

## 2.2 Parâmetros do experimentos

In [50]:
#nível de confiança
confidence_level = 0.95

#nível de significancia
significance_level = 1 - confidence_level

#conversões da página atual e da nova página
p1 = 0.13 #atual
p2 = 0.15 #novo

#tamanho do efeito
effect_size = sms.proportion_effectsize(p1,p2)

#poder estatístico
power = 0.8

In [51]:
#sample size - Tamanho da amostra
sample_n = sms.NormalIndPower().solve_power(
  effect_size,
  power=power,
  alpha=significance_level
)

sample_n = math.ceil(sample_n)
print(f'O tamanho da amostra do grupo controle é de {sample_n}.')
print(f'O tamanho da amostra do grupo de tratamento é de {sample_n}.')
print(f'O tamanho total da amostra é de {sample_n*2}.')

O tamanho da amostra do grupo controle é de 4720.
O tamanho da amostra do grupo de tratamento é de 4720.
O tamanho total da amostra é de 9440.


#Preparar os dados

#Amostragem

#Taxa de conversão

#Teste de Hipóteses

#Conclusão

#Conversão de resultado para R$

# 3.0 Análise descritiva dos dados

In [52]:
print(f'Number of rows: {df_raw.shape[0]}')
print(f'Number of columns: {df_raw.shape[1]}')

Number of rows: 294478
Number of columns: 5


## 3.1 Verificação dos dados faltantes

In [53]:
df_raw.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

## 3.2 Conferir as 'flags'

In [54]:
df_raw.loc[:,['user_id', 'group', 'landing_page']].groupby(['group', 'landing_page']).count().reset_index()

,group,landing_page,user_id
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


## 3.4 Quantos usurários estão em mais de um grupo?

In [55]:
df_raw.loc[:,['user_id', 'group']].groupby('user_id').count().sort_values(by = 'group',ascending=False).reset_index().query('group > 1')

,user_id,group
0,729669,2
1,825266,2
2,663033,2
3,674959,2
4,875210,2
...,...,...
3889,800469,2
3890,738614,2
3891,651850,2
3892,720460,2


Vamos deletar os valores pois eles representam menos de 1% da base

In [56]:
user_delete = df_raw.loc[:,['user_id', 'group']].groupby('user_id').count().sort_values(by = 'group',ascending=False).reset_index().query('group > 1')['user_id']

df1 = df_raw[~df_raw['user_id'].isin(user_delete)]
df1.shape

(286690, 5)

## 3.4 Conferir as 'flags' depois de tirar os duplicados

In [57]:
df1.loc[:,['user_id', 'group', 'landing_page']].groupby(['group', 'landing_page']).count().reset_index()

,group,landing_page,user_id
0,control,old_page,143293
1,treatment,new_page,143397


## 3.5 Amostragem aleatória dos grupos de Controle e Tratamento

In [58]:
#Grupo Controle
df_control_sample = df1.loc[df1['group'] == 'control',:].sample(sample_n,random_state=42)
print(f'Size of Control Group: {df_control_sample.shape[0]}')

#Grupo Tratamento
df_treatment_sample = df1.loc[df1['group'] == 'treatment',:].sample(sample_n,random_state=42)
print(f'Size of Treatment Group: {df_treatment_sample.shape[0]}')

#Total sample_size
df_ab = pd.concat([df_treatment_sample,df_control_sample]).reset_index(drop=True)

Size of Control Group: 4720
Size of Treatment Group: 4720


## 3.6 Conversão das páginas

In [59]:
# Cálculo da métrica de interesse entre os grupos (Conversão da Página)
# Conversão = compradores/visitantes
# Grupo Controle
sales_c = df_control_sample.loc[df_control_sample['converted'] == 1,'converted'].sum()
visitors_c = df_control_sample.loc[:,'user_id'].count()

conversion_rate_control = sales_c/visitors_c
print(f'Conversion Rate = Control Group: {round(conversion_rate_control*100,2)}%')

# Grupo de Tratamento
sales_t = df_treatment_sample.loc[df_treatment_sample['converted'] == 1,'converted'].sum()
visitors_t = df_treatment_sample.loc[:,'user_id'].count()

conversion_rate_treatment = sales_t/visitors_t
print(f'Conversion Rate = Treatment Group: {round(conversion_rate_treatment*100,2)}%')

Conversion Rate = Control Group: 11.55%
Conversion Rate = Treatment Group: 12.9%


## 3.7 Teste de Hipóteses

In [60]:
#Teste de Hipóteses
df_table = df_ab[['group','converted']].groupby('group').agg({'converted': ['sum','count']})
df_table.columns = ['converted','total']

chi_val,pval,dof,expected = chi2_contingency(df_table)
print(f'p-value:{round(pval,2)}')

if pval < significance_level:
  print(f'Rejeita a hipótese nula')
else:
  print(f'Falha em rejeitar a hipótese nula')

p-value:0.08
Falha em rejeitar a hipótese nula


In [16]:
df_table.head()

,converted,total
group,,
control,545,4720
treatment,609,4720


# 4.0 Conversão da Página em Faturamento

In [18]:
df4 = df1.copy()

df4['timestamp'] = pd.to_datetime(df4['timestamp']).apply(lambda x: x.strftime('%Y-%m-%d'))
df4.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21,control,old_page,0
1,804228,2017-01-12,control,old_page,0
2,661590,2017-01-11,treatment,new_page,0
3,853541,2017-01-08,treatment,new_page,0
4,864975,2017-01-21,control,old_page,1


In [19]:
##número de visitantes da página por dia
df5 = df4[['user_id','timestamp']].groupby('timestamp').count().reset_index()

#Current GMV
df5['current_purchases'] = np.ceil(df5['user_id'] * 0.13).astype(int)
df5['current_GMV'] = df5['current_purchases'] * 4500

current_gmv = df5['current_GMV'].sum()
print(f'GMV On Period:{current_gmv}')

#Expected GMV
df5['new_purchases'] = np.ceil(df5['user_id'] * 0.15).astype(int)
df5['new_GMV'] = df5['new_purchases'] * 4500

new_gmv = df5['new_GMV'].sum()
print(f'New GMV On Period:{new_gmv}')

#Diferença
lift_abs = new_gmv-current_gmv
print(f'Abs Lift:{lift_abs}')

#Variação
lift = (new_gmv-current_gmv)/current_gmv
print(f'Expected Lift:{round(lift*100,2)}%')

GMV On Period:167760000
New GMV On Period:193563000
Abs Lift:25803000
Expected Lift:15.38%
